<a href="https://colab.research.google.com/github/azzindani/01_Data_Analytic/blob/main/US_Unemployment_Draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 00 Importing Modules & Mounting Drive

In [ ]:
!pip install pandas-profiling
!pip install streamlit
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.655s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
import pathlib
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from ydata_profiling import ProfileReport
from tabulate import tabulate
from time import sleep

%matplotlib inline

In [ ]:
'''from google.colab import drive
drive.mount('/content/drive') #'''

"from google.colab import drive\ndrive.mount('/content/drive') #"

In [ ]:
'''MAIN_PATH = str(pathlib.Path().resolve())
WORK_PATH = MAIN_PATH + '/drive/MyDrive/Workspace'
SOURCE_PATH = WORK_PATH + '/00_Data_Source'
CACHE_PATH = WORK_PATH + '/00_Cache_Data' #'''

"MAIN_PATH = str(pathlib.Path().resolve())\nWORK_PATH = MAIN_PATH + '/drive/MyDrive/Workspace'\nSOURCE_PATH = WORK_PATH + '/00_Data_Source'\nCACHE_PATH = WORK_PATH + '/00_Cache_Data' #"

## 01 Choosing & Importing Dataset

### 01.01 Importing Main Data

In [ ]:
dataset_path = 'https://raw.githubusercontent.com/azzindani/00_Data_Source/main/US_Unemployment.csv'
df = pd.read_csv(dataset_path)
df.head(2)

,index,Month and Year,State,Civilian Population,Total Labor Force,Percent of Population,Employed - Total,Employed - Percent of Population,Unemployed - Total,Unemployed - Percent of Labor Force
0,0,January 1976,Alabama,2605000,1492409,57.3,1392154,53.4,100255,6.7
1,1,January 1976,Alaska,232000,159154,68.6,147809,63.7,11345,7.1


### 01.02 Importing Geo Data

In [ ]:
gdf = gpd.read_file('https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json')
gdf.head(2) #'''

,id,name,density,geometry
0,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984..."
1,02,Alaska,1.264,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691..."


In [ ]:
gdf = gdf.rename(columns = {'name' : 'State'})
gdf = gdf[['State', 'geometry']]
gdf.head(2)

,State,geometry
0,Alabama,"POLYGON ((-87.35930 35.00118, -85.60667 34.984..."
1,Alaska,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691..."


### 01.03 Importing Additional Data (for enrichment)

## 02 Checking Data

In [ ]:
df.shape

(26316, 10)

In [ ]:
df.columns

Index(['index', 'Month and Year', 'State', 'Civilian Population',
       'Total Labor Force', 'Percent of Population', 'Employed - Total',
       'Employed - Percent of Population', 'Unemployed - Total',
       'Unemployed - Percent of Labor Force'],
      dtype='object')

In [ ]:
df.nunique()

index                                  26316
Month and Year                           516
State                                     51
Civilian Population                    22345
Total Labor Force                      26203
Percent of Population                    252
Employed - Total                       26211
Employed - Percent of Population         303
Unemployed - Total                     24911
Unemployed - Percent of Labor Force      153
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26316 entries, 0 to 26315
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   index                                26316 non-null  int64  
 1   Month and Year                       26316 non-null  object 
 2   State                                26316 non-null  object 
 3   Civilian Population                  26316 non-null  int64  
 4   Total Labor Force                    26316 non-null  int64  
 5   Percent of Population                26316 non-null  float64
 6   Employed - Total                     26316 non-null  int64  
 7   Employed - Percent of Population     26316 non-null  float64
 8   Unemployed - Total                   26316 non-null  int64  
 9   Unemployed - Percent of Labor Force  26316 non-null  float64
dtypes: float64(3), int64(5), object(2)
memory usage: 2.0+ MB


## 03 Data Structuring

### 03.01 Selecting & Dropping Variables

### 03.02 Cleaning Text Object

#### 03.02.01 Convert Header to Proper Text

In [ ]:
for x in df.columns:
  y = x.title()
  df = df.rename(columns = {x : y})

#### 03.02.02 Strip Abnormal Spaces

In [ ]:
for x in df.columns:
  if df[x].dtypes == 'object':
    try:
      df[x] = df[x].str.strip()
    except:
      pass

#### 03.02.03 Convert Object Content to Proper Text

In [ ]:
for x in df.columns:
  if df[x].dtypes == 'object':
    for a in df[x].unique():
      b = a.title()
      df[x] = df[x].replace(a, b)
  else:
    pass

### 03.03 Coverting Data

#### 03.03.01 Convert to date

In [ ]:
df['Month And Year'] = df['Month And Year'].replace({
    'January ' : '31-01-',
    'February ' : '28-02-',
    'March ' : '31-03-',
    'April ' : '30-04-',
    'May ' : '31-05-',
    'June ' : '30-06-',
    'July ' : '31-07-',
    'August ' : '31-08-',
    'September ' : '30-09-',
    'October ' : '31-10-',
    'November ' : '30-11-',
    'December ' : '31-12-',
}, regex = True)

In [ ]:
df['Month And Year'] = pd.to_datetime(df['Month And Year'])

<ipython-input-16-7af147c2c3d9>:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Month And Year'] = pd.to_datetime(df['Month And Year'])


#### 03.03.02 Convert to integer

#### 03.03.03 Convert to Object (if necessary)

#### 03.03.04 Replace 0 to Nan (if necessary)

In [ ]:
#df = df.replace(0, np.nan)

## 04 Data Cleaning

### 04.01 Replacing Variable Component

### 04.02 Add New Variable

### 04.03 Inaccuracies

### 04.04 Handling Outliers
##### Data Distribution Check (Before)

In [ ]:
df.head(2)

,Index,Month And Year,State,Civilian Population,Total Labor Force,Percent Of Population,Employed - Total,Employed - Percent Of Population,Unemployed - Total,Unemployed - Percent Of Labor Force
0,0,1976-01-31,Alabama,2605000,1492409,57.3,1392154,53.4,100255,6.7
1,1,1976-01-31,Alaska,232000,159154,68.6,147809,63.7,11345,7.1


#### 04.04.01 Using IQR (Inter Quantile Range)

In [ ]:
def iqr_thres(dataframe, column, th1 = 0.25, th3 = 0.75):
    quartile1 = dataframe[column].quantile(th1)
    quartile3 = dataframe[column].quantile(th3)
    iqr = quartile3 - quartile1
    upper_limit = quartile3 + 1.5 * iqr
    lower_limit = quartile1 - 1.5 * iqr

    return lower_limit, upper_limit

In [ ]:
def check_outliers_iqr(dataframe, column):
    lower_limit, upper_limit = iqr_thres(dataframe, column)
    if dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)].any(axis = None):
        return True
    else:
        return False

In [ ]:
def replace_iqr(dataframe, columns, th1 = 0.25, th3 = 0.75, replace = False):
  data = []

  for column in columns:
    if df[column].dtypes == 'int64' or df[column].dtypes == 'float64':
      if column != 'Outcome':
        outliers_ = check_outliers_iqr(df, column)
        count = None
        lower_limit, upper_limit = iqr_thres(dataframe, column, th1, th3)

        if outliers_:
          count = dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)][column].count()
          if replace:
            if lower_limit < 0:
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
            else:
              dataframe.loc[(dataframe[column] < lower_limit), column] = lower_limit
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
        outliers_status = check_outliers_iqr(df, column)
        data.append([outliers_, outliers_status, count, column, lower_limit, upper_limit ])

  table = tabulate(data, headers = ['Outliers (Previously)', 'Outliers', 'Count', 'Column', 'Lower Limit', 'Upper Limit'], tablefmt = 'rst', numalign = 'right')
  print('Removing Outliers using IQR')
  print(table)

In [ ]:
'''replace_iqr(
    dataframe = df,
    columns = [],
    replace = True
) #'''

'replace_iqr(\n    dataframe = df,\n    columns = [],\n    replace = True\n) #'

#### 04.04.02 Using Standard Deviation

In [ ]:
def std_thres(dataframe, column):
    upper_limit = dataframe[column].mean() + 3 * dataframe[column].std()
    lower_limit = dataframe[column].mean() - 3 * dataframe[column].std()

    return lower_limit, upper_limit

In [ ]:
def check_outliers_std(dataframe, column):
    lower_limit, upper_limit = iqr_thres(dataframe, column)
    if dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)].any(axis = None):
        return True
    else:
        return False

In [ ]:
def replace_std(dataframe, columns, replace = False):
  data = []

  for column in columns:
    if df[column].dtypes == 'int64' or df[column].dtypes == 'float64':
      if column != 'Outcome':
        outliers_ = check_outliers_std(dataframe, column)
        count = None
        lower_limit, upper_limit = std_thres(dataframe, column)

        if outliers_:
          count = dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)][column].count()
          if replace:
            if lower_limit < 0:
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
            else:
              dataframe.loc[(dataframe[column] < lower_limit), column] = lower_limit
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
        outliers_status = check_outliers_std(dataframe, column)
        data.append([outliers_, outliers_status, count, column, lower_limit, upper_limit])

  table = tabulate(data, headers = ['Outlier (Previously)', 'Outliers', 'Count', 'Column', 'Lower Limit', 'Upper Limit'], tablefmt = 'rst', numalign = 'right')
  print('Removing Outliers using 3 Standard Deviation')
  print(table)

In [ ]:
'''replace_iqr(
    dataframe = df,
    columns = [],
    replace = False
) #'''

'replace_iqr(\n    dataframe = df,\n    columns = [],\n    replace = False\n) #'

### 04.05 Handling Missing / Zeros / Null
##### Filling missing value (numerical only) is better using median than mean or mode

#### 04.05.01 Detecting Zero Value

In [ ]:
zero_columns = []

for x in df.columns:
  if df[x].dtypes == 'int64' or df[x].dtypes == 'float64':
    if (df[x] == 0).sum() != 0:
      print(x, ':', str((df[x] == 0).sum()))
      zero_columns.append(x)

Index : 1


#### 04.05.02 Detecting Nan / Non Available Value

In [ ]:
nan_columns = []

for x in df.columns:
  if df[x].dtypes == 'object':
    if (df[x] == np.nan).sum() or (df[x] == '-').sum() != 0:
      print(x, ':', str((df[x] == np.nan).sum()))
      nan_columns.append(x)

#### 04.05.03 Replacing Zero with Mean (for numerical value if median value == 0), if necessary

In [ ]:
'''cols = [
] #'''

'cols = [\n] #'

In [ ]:
'''df = df.replace(0, np.nan)

for x in cols:
  df[x] = df[x].fillna(df[x].mean()) #'''

'df = df.replace(0, np.nan)\n\nfor x in cols:\n  df[x] = df[x].fillna(df[x].mean()) #'

#### 04.05.04 Replacing Zero with Median (for numerical value if median value != 0), if necessary

In [ ]:
'''cols = [
] #'''

'cols = [\n] #'

In [ ]:
'''df = df.replace(0, np.nan)

for x in cols:
  df[x] = df[x].fillna(df[x].median()) #'''

'df = df.replace(0, np.nan)\n\nfor x in cols:\n  df[x] = df[x].fillna(df[x].median()) #'

#### 04.05.05 Replacing Zero with Mode (for categorical / object value), if necessary

In [ ]:
'''cols = [
] #'''

'cols = [\n] #'

In [ ]:
'''df = df.replace(0, np.nan)

for x in cols:
  df[x] = df[x].fillna(df[x].mode()) #'''

'df = df.replace(0, np.nan)\n\nfor x in cols:\n  df[x] = df[x].fillna(df[x].mode()) #'

### 04.06 Handling Incomplete Data

### 04.07 Handling Data Biases

### 04.08 Handling Duplicates

In [ ]:
#df = df.drop_duplicates()

##### Data Distribution Check (After)

## 05 Enriching Data
#### take other dataset, inside or outside from related dataset / business

In [ ]:
a = df['State'].unique()
b = gdf['State'].unique()

for i in a:
  if i not in b:
    print(i)

print(50 * '=')

for i in b:
  if i not in a:
    print(i)

District Of Columbia
District of Columbia
Puerto Rico


In [ ]:
df['State'] = df['State'].replace({
    'District Of Columbia' : 'District of Columbia',
})

## 06 Data Validation
#### Verifying consistency, quality, and security of data

## 07 Exploration Data Analysis (Univariate)

### 07.01 Generate Data Profile

In [ ]:
'''data_profile = ProfileReport(
    df,
    title = 'US Unemployment',
    dataset = {
        #'description' : 'This profiling report was generated for Analytics Vidhya Blog',
        #'copyright_holder' : 'Analytics Vidhya',
        #'copyright_year' : '2021',
        #'url' :  'https://www.analyticsvidhya.com/blog/',
    },
    correlations = {
        'pearson' : {'calculate' : True},
        'spearman' : {'calculate' : True},
        'kendall' : {'calculate' : True},
        'phi_k' : {'calculate' : True},
    },
)

data_profile #'''

"data_profile = ProfileReport(\n    df,\n    title = 'US Unemployment',\n    dataset = {\n        #'description' : 'This profiling report was generated for Analytics Vidhya Blog',\n        #'copyright_holder' : 'Analytics Vidhya',\n        #'copyright_year' : '2021',\n        #'url' :  'https://www.analyticsvidhya.com/blog/',\n    },\n    correlations = {\n        'pearson' : {'calculate' : True},\n        'spearman' : {'calculate' : True},\n        'kendall' : {'calculate' : True},\n        'phi_k' : {'calculate' : True},\n    },\n)\n\ndata_profile #"

## 08 Store Data to Google Drive

In [ ]:
'''cache_dir = CACHE_PATH + '/_cache.csv'
print(cache_dir)
df.to_csv(cache_dir, index = False)

sleep(60) #'''

"cache_dir = CACHE_PATH + '/_cache.csv'\nprint(cache_dir)\ndf.to_csv(cache_dir, index = False)\n\nsleep(60) #"

## 09 Table, Chart Drafting & Testing

In [ ]:
fig = px.colors.sequential.swatches_continuous()
#fig.show()

In [ ]:
colors_1 = px.colors.sequential.Reds
colors_2 = px.colors.sequential.RdBu
explode = tuple([0.015] * 50)
latitude = 57
longitude = -113
chart_theme = 'plotly_dark'
streamlit_theme = 'streamlit'
margin = {'r' : 20, 't' : 40, 'l' : 20, 'b' : 10}
cmap = 'magma_r'
title_x = 0
title_font_size = 18

In [ ]:
var_num = 'Civilian Population'
df = df[['State', 'Month And Year', var_num]]
df.head(2)

,State,Month And Year,Civilian Population
0,Alabama,1976-01-31,2605000
1,Alaska,1976-01-31,232000


In [ ]:
df['Month'] = df['Month And Year'].dt.to_period('M')
linechart = pd.DataFrame(df.groupby(['Month'])[[var_num]].sum()).reset_index()
linechart['Month'] = linechart['Month'].astype(str)

# create time series chart

title = 'Monthly ' + var_num
#st.subheader(title)
fig_1 = go.Figure()
fig_1.add_trace(go.Scatter(
  x = linechart['Month'],
  y = linechart[var_num],
  mode = 'lines+markers',
  marker = {'color': colors_2[0]},
))
fig.update_traces(
  hovertemplate = '{:,.0}'
)
fig_1.update_layout(
  hovermode = 'x',
  height = 400,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
  yaxis = dict(tickformat = ',.0f'),
)
fig_1.show()

<ipython-input-43-aa49dcead3c1>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df['Year'] = df['Month And Year'].dt.to_period('Q')
linechart = pd.DataFrame(df.groupby(['Year', 'State'])[[var_num]].sum()).reset_index()
linechart['Year'] = linechart['Year'].astype(str)

fig_1 = px.line(
  x = linechart['Year'],
  y = linechart[var_num],
  color = linechart['State'],
  markers = True,
)
fig_1.update_traces(
  #hovertemplate = '{:,.0f}'
)
fig_1.update_layout(
  hovermode = 'x',
  height = 400,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
  yaxis = dict(tickformat = ',.0f'),
)
fig_1.show()

In [ ]:
chart_df_3 = df.groupby(by = ['State'], as_index = False,)[[var_num]].sum()
chart_df_3 = chart_df_3.sort_values(by = var_num, ascending = False)[:10]

title = '10 Highest ' + var_num + ' State'
#st.subheader(title)
fig = px.bar(
  chart_df_3,
  x = var_num,
  y = 'State',
  template = chart_theme,
  color_discrete_sequence = colors_2,
  title = title,
  text_auto = ',.0f',
)
fig.update_layout(
    height = 400,
    margin = margin,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
)
fig.update_yaxes(categoryorder = 'total ascending')
fig.show()

In [ ]:
var_label = 'State'
var_number = var_num

map_df = df.groupby(['State'])[var_num].sum()
map_df = map_df.reset_index()
map_df = map_df.fillna(0)
map_df = gdf.merge(map_df, on = 'State')

map_df = map_df.set_index(var_label)
map_df.head(2)

,geometry,Civilian Population
State,,
Alabama,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1698463101
Alaska,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691...",214652091


In [ ]:
fig = px.choropleth_mapbox(
    data_frame = map_df,
    geojson = map_df.geometry,
    locations = map_df.index,
    color = var_number,
    color_continuous_scale = colors_1,
    #range_color = (0, 10),
    opacity = 0.5,
    center = {'lat' : latitude, 'lon' : longitude},
    mapbox_style = 'carto-positron',
    zoom = 2,
    #width = 800,
    height = 600,
)

fig.update_geos(fitbounds = 'locations', visible = False)
fig.update_layout(margin = {'r' : 0, 't' : 0, 'l' : 0, 'b' : 0},)

fig.show()

## 10 Create Dashboard
### * template : ['ggplot2', 'seaborn', 'simple_white', 'plotly', 'plotly_white', 'plotly_dark', 'presentation', 'xgridoff', 'ygridoff', 'gridon', 'none']
### * change layout https://plotly.com/python-api-reference/generated/plotly.graph_objects.Layout.html
### * more chart customization : https://plotly.com/python-api-reference/index.html

In [ ]:
%%writefile app.py

import streamlit as st
import os
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import warnings
warnings.filterwarnings('ignore')

# 00 CREATING TAB TITLE

st.set_page_config(
  page_title = 'US Unemployment',
  page_icon = ':male-office-worker:',
  layout = 'wide',
)


# 01 CREATING DASHBOARD TITLE

st.title(':male-office-worker: US Unemployment Dashboard')
st.markdown('<style>div,block-container{padding-top:0rem;}<style>', unsafe_allow_html = True)


# 02 IMPORTING DATASET

dataset_path = 'https://raw.githubusercontent.com/azzindani/00_Data_Source/main/US_Unemployment.csv'
df = pd.read_csv(dataset_path)
df = df.fillna(0)

gdf = gpd.read_file('https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json')
gdf = gdf.rename(columns = {'name' : 'State'})
gdf = gdf[['State', 'geometry']]

for x in df.columns:
  y = x.title()
  df = df.rename(columns = {x : y})

for x in df.columns:
  if df[x].dtypes == 'object':
    try:
      df[x] = df[x].str.strip()
    except:
      pass

for x in df.columns:
  if df[x].dtypes == 'object':
    for a in df[x].unique():
      b = a.title()
      df[x] = df[x].replace(a, b)
  else:
    pass

df['Month And Year'] = df['Month And Year'].replace({
    'January ' : '31-01-',
    'February ' : '28-02-',
    'March ' : '31-03-',
    'April ' : '30-04-',
    'May ' : '31-05-',
    'June ' : '30-06-',
    'July ' : '31-07-',
    'August ' : '31-08-',
    'September ' : '30-09-',
    'October ' : '31-10-',
    'November ' : '30-11-',
    'December ' : '31-12-',
}, regex = True)

df['Month And Year'] = pd.to_datetime(df['Month And Year'])
df['Year'] = df['Month And Year'].dt.to_period('Y')
df['Year'] = df['Year'].astype('str')
df['Year'] = df['Year'].astype('int')

df['State'] = df['State'].replace({
    'District Of Columbia' : 'District of Columbia',
})


# 03 SETUP TEMPLATE & THEME

colors_1 = px.colors.sequential.Blues
colors_2 = px.colors.sequential.Blues_r
explode = tuple([0.015] * 50)
latitude = 57
longitude = -113
chart_theme = 'plotly_dark'
streamlit_theme = 'streamlit'
margin = {'r' : 20, 't' : 40, 'l' : 20, 'b' : 10}
cmap = 'Blues'
title_x = 0
title_font_size = 18


col_1, col_2 = st.columns((1, 1))

# 05 CREATING FILTER

var_nums = df.columns[3:]

with col_1:
  var_num = st.selectbox('Select Variable', var_nums)

with col_2:
  year = st.slider(
    label = 'Select Year',
    min_value = df['Year'].min(),
    max_value = df['Year'].max(),
    value = df['Year'].max(),
  )

df = df[['State', 'Month And Year', 'Year', var_num]]
filtered_df = df[df['Year'] == year]


# 07 CREATING DASHBOARD

# create highlighted indicator

col_11, col_12 = st.columns((1, 2))

chart_df_1 = filtered_df.groupby(by = ['State'], as_index = False,)[[var_num]].sum()
chart_df_1 = chart_df_1.sort_values(by = var_num, ascending = False)[:10]

with col_11:
  title = '10 Highest ' + var_num + ' State'
  #st.subheader(title)
  fig = px.bar(
    chart_df_1,
    x = var_num,
    y = 'State',
    template = chart_theme,
    color_discrete_sequence = colors_2,
    title = title,
    text_auto = ',.0f',
  )
  fig.update_layout(
      height = 600,
      margin = margin,
      titlefont = dict(size = title_font_size),
      title_x = title_x,
  )
  fig.update_yaxes(categoryorder = 'total ascending')
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_1.style.background_gradient(cmap = cmap))
    csv = chart_df_1.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

var_label = 'State'
var_number = var_num

map_df = filtered_df.groupby(['State'])[var_num].sum()
map_df = map_df.reset_index()
map_df = map_df.fillna(0)
map_df = gdf.merge(map_df, on = 'State')
map_df = map_df.set_index(var_label)

with col_12:
  fig = px.choropleth_mapbox(
      data_frame = map_df,
      geojson = map_df.geometry,
      locations = map_df.index,
      color = var_number,
      color_continuous_scale = colors_1,
      #range_color = (0, 10),
      opacity = 0.5,
      center = {'lat' : latitude, 'lon' : longitude},
      mapbox_style = 'carto-positron',
      zoom = 2,
      #width = 800,
      height = 600,
  )
  fig.update_geos(
      #fitbounds = 'locations',
      visible = False,
  )
  fig.update_layout(
      height = 600,
      margin = margin,
      title = title,
      titlefont = dict(size = title_font_size),
      title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

# create time series chart

col_11, col_12 = st.columns((1, 1))

default_state = 'Florida'

with col_11:
  state = st.multiselect('Select State', df['State'].unique(), default = default_state)

if state:
  state_df = df[df['State'].isin(state)]
else:
  state_df = df

periods = ['M', 'Q', 'Y']

with col_12:
  period = st.selectbox('Select Period (M = Month, Q = Quarter, Y = Year)', periods)

title = 'Periodly ' + var_num

state_df['Period'] = state_df['Month And Year'].dt.to_period(period)
linechart = pd.DataFrame(state_df.groupby(['Period', 'State'])[[var_num]].sum()).reset_index()
linechart['Period'] = linechart['Period'].astype(str)

fig_1 = px.line(
  x = linechart['Period'],
  y = linechart[var_num],
  color = linechart['State'],
  markers = True,
)
fig_1.update_traces(
  #hovertemplate = '{:,.0f}'
)
fig_1.update_layout(
  hovermode = 'x',
  height = 400,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
  xaxis = dict(title = 'Period', titlefont = dict(size = 14)),
  yaxis = dict(title = var_num, titlefont = dict(size = 14), tickformat = ',.0f'),
)

st.plotly_chart(fig_1, use_container_width = True, theme = streamlit_theme)

with st.expander('View Data'):
  st.write(linechart.style.background_gradient(cmap = cmap))
  csv = linechart.to_csv(index = False).encode('utf-8')
  st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

Overwriting app.py


## 11 Run Dashboard

In [ ]:
!streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com
!npx localtunnel --port 8501

35.202.200.51
npx: installed 22 in 2.67s
your url is: https://spicy-hoops-care.loca.lt
^C
